In [1]:
import networkx as nx

In [3]:
G2 = nx.read_graphml('test2.graphml')

In [7]:
% matplotlib inline

In [8]:
from bokeh.models import ColumnDataSource
from math import sqrt
from bokeh.plotting import show, figure
from bokeh.io import output_notebook
from bokeh.models import HoverTool

def get_edges_specs(_network, _layout):
    d = dict(xs=[], ys=[], alphas=[])
    #weights = [d['weight'] for u, v, d in _network.edges(data=True)]
    max_weight = 1 #max(weights)
    calc_alpha = lambda h: 0.1 + 0.6 * (h / max_weight)

    # example: { ..., ('user47', 'da_bjoerni', {'weight': 3}), ... }
    for u, v, data in _network.edges(data=True):
        d['xs'].append([_layout[u][0], _layout[v][0]])
        d['ys'].append([_layout[u][1], _layout[v][1]])
        d['alphas'].append(calc_alpha(data['weight']))
    return d
K = .1/sqrt(G2.number_of_nodes())
layout = nx.spring_layout(G2,k=K,iterations=100)
nodes, nodes_coordinates = zip(*sorted(layout.items()))
nodes_xs, nodes_ys = list(zip(*nodes_coordinates))
colors = tuple([G2.node[node]['color'] for node in nodes])
lbl = tuple([G2.node[node]['lbl'] for node in nodes])
nodes_source = ColumnDataSource(dict(x=nodes_xs, y=nodes_ys,
                                     name=nodes,color=colors,label=lbl))


hover = HoverTool(tooltips=[('label', '@label')])
plot = figure(plot_width=800, plot_height=800,
              tools=['tap', hover, 'box_zoom', 'reset'])
r_circles = plot.circle('x', 'y', source=nodes_source, size=10,color=colors,level='overlay')
                        #color='blue', level = 'overlay')



lines_source = ColumnDataSource(get_edges_specs(G2, layout))
r_lines = plot.multi_line('xs', 'ys', line_width=1.5,
                          alpha='alphas', color='navy',
                          source=lines_source)
output_notebook(); show(plot)

Loading BokehJS ...